---
<center><h1> Lesson 8.1 - Work with Twitter API in Python</center></h1>

---

A Web API is an *application programming interface* (**API**). It helps easely to entaract with program programs and web services.  

Today many services provide its API for developers (such as Facebook, Google, Twitter, etc). API helps to en to interact with their services and to access data in a programmatic way. With help of API, you can create your own application.  

API is a set of subroutine definitions, protocols, and tools for building application software. In general terms, it is a set of clearly defined methods of communication between various software components. A good API makes it easier to develop a computer program by providing all the building blocks, which are then put together by the programmer. An API may be for a web-based system, operating system, database system, computer hardware or software library.  

The current lesson is devoted to Twitter API. We will consider both approaches of getting access to Twitter API - main Python tools such as `request` library and specific Python libraries such as `tweepy`.

<img src="images/Python_Twitter.jpg" width="512px" height="288px">

One of the reasons that Twitter is so popular, and become what it is today, is due to the huge number of developers who have built apps over the years. Why so many developers? Basically because Twitter has been very easy to work with. Twitter uses the OAuth protocol for secure application development. 

In this lesson, you will create an application integrated with Twitter. Registering the application is the first step in developing an application that is integrated into its users' social graph.

## Register a new application
All Twitter users are potentially Twitter application developers. Visit https://apps.twitter.com and and press "Create New App".   

<img src="images/twitter_01.jpg">

After that fill Aplication Detals (Name, Description and Website), read and agree to the Twitter Developer Agreement. Press "Create your Twitter application".  
(Note that you must add your mobile phone to your Twitter profile before creating an application. Please read [here](https://support.twitter.com/articles/110250-adding-your-mobile-number-to-your-account-via-web) for more information.)

<img src="images/twitter_02.jpg">  
After Captcha challenge click "Create your Twitter application" to complete the form and go to the application settings page.

We now need to authorise the Twitter app for your Twitter account. To do this you should select "Key and Access Tokens" menu button and create your access token. This **access token** will allow your Twitter application to read Twitter information. You’ll be able to get data of your tweets, mentions, lists and more.  

<img src="images/twitter_03.jpg">

In the opened window you’ll then be presented with lots of information. The main fields to note are **_Consumer key_** and **_Consumer secret_**. These values are your application’s credentials for Twitter. You need them to do almost anything with Twitter, including going through the OAuth authorization flow and working with Twitter’s REST API. But we’re not quite done yet. 

As a result you will see the window with consumer and token keys for your application 

<img src="images/twitter_04.jpg">

Above the four fields are highlighted. You will need these long horrible strings of characters for your Twitter app. 

---
## 1. Getting access to Twitter APIs using main Python tools
---

Many web services including Twitter require authentication, and there are many different types. 

Python library `requests` provides a pythonic way to make complex HTTP requests, and handles difficult tasks like authentication.

A common form of authentication for several web APIs is OAuth. The `requests-oauthlib` library allows `requests`'s users to easily make OAuth authenticated requests.

In [2]:
import requests
from requests_oauthlib import OAuth1
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv()

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = os.environ['consumer_key']
consumer_secret = os.environ['consumer_secret']

access_token = os.environ['access_token']
access_secret = os.environ['access_secret']

# You will authenticate yourself using OAuth1 object
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print (auth)

In [3]:
# If the authentication was successful, you should see the name of the account print out
url_0 = 'https://api.twitter.com/1.1/account/verify_credentials.json'
res = requests.get(url_0, auth=auth)

print ("My name is", res.json()['name']) 

My name is Dmitriy Kisil


### 1.1: Read Tweets from Your Homepage

The list of all available Twitter API you can find here https://developer.twitter.com/en/docs/api-reference-index. Let's use `home_timeline` that returns a collection of the most recent tweets and retweets posted by the authenticating user and the users they follow.

In [4]:
url_1 = 'https://api.twitter.com/1.1/statuses/home_timeline.json'
res = requests.get(url_1, auth=auth, params={"count": 3})

# The res object encapsulates the "response" of the server. Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 403 means that the server understood the request, but is refusing to fulfill it
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong

print (res, "Status code:", res.status_code)
# Let's see how looks the url
print (res.url)
print ("Content type:", res.headers['content-type'], '\n')

tweets = res.json()
if res.status_code == 200:
    for tweet in tweets:
        print (tweet['text'])
else:   # You have no tweets
    print (tweets)

<Response [200]> Status code: 200
https://api.twitter.com/1.1/statuses/home_timeline.json?count=3
Content type: application/json;charset=utf-8 

Закликаю українських громадян на окупованих Росією територіях не приймати російські паспорти. Росія позбавила вас с… https://t.co/MMmFo5kAAN
Рішення РФ щодо видачі 🇷🇺паспортів на окупованих українських територіях - це продовження агресії і втручання у наші… https://t.co/tgbCADMSP7
Hacking Neural Networks by @mrdragonbear https://t.co/FQPYx17VfX


### 1.2: Search tweets by key words

Twitter employs a special query language.  For example, the query _"luck?"_ will return tweets that contain the word "luck" and are phrased as a question OR the query _"winter"_ will return the word "winter" with a positive attitude. [Check out more examples here](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets).

We can mine tweets using either **_search_** or **_stream_**.

>The key difference between **_stream_** and **_search_** is that **_stream_** provides new data as it comes in, while **_search_** can be used to query old data. The **_search_** API is more powerful for queries, and provides faster access to a wide-range of data.

Let's search for a single tweet about "luck?", phrased as a question. Note if request phrase contains URI characters you should [encode](https://en.wikipedia.org/wiki/Percent-encoding) these queries before making the request, because these characters can play role of keys in an URL. Particularly, the question mark "?" is encoded as "%3F".

In [5]:
url_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=luck%3F'
res = requests.get(url_2, auth=auth)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

tweets = res.json()
tweets

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=luck%3F


{'statuses': [{'created_at': 'Wed Apr 24 13:20:39 +0000 2019',
   'id': 1121041263636176897,
   'id_str': '1121041263636176897',
   'text': 'RT @OfficialMazzuma: Hello there,\n\nWe are looking for a marketing associate. Think you have what it takes to join our team?\n\nThen check out…',
   'truncated': False,
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [{'screen_name': 'OfficialMazzuma',
      'name': 'Mazzuma',
      'id': 3165440693,
      'id_str': '3165440693',
      'indices': [3, 19]}],
    'urls': []},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to_user_id': None,
   'in_reply_to_user_id_str': None,
   'in_reply_to_screen_name': None,
   'user': {'id': 1004774059291312129,
    'id_str': '1004774059291312129',
    'name': 'sami_danso',
    'scre

Please pay your attention how many information each tweet contains.

In [6]:
# And let's see tweets text directly
for num, tweet in enumerate(tweets['statuses']):
    print ('Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8")))

Tweet #1	Wed Apr 24 13:20:39 +0000 2019
b'RT @OfficialMazzuma: Hello there,\n\nWe are looking for a marketing associate. Think you have what it takes to join our team?\n\nThen check out\xe2\x80\xa6'

Tweet #2	Wed Apr 24 13:20:38 +0000 2019
b'RT @HJCRecruitment: Have you entered our #FreebieFriday #Competition yet folks? \xf0\x9f\x91\x8f\xf0\x9f\x8f\xbc\n\nJust FOLLOW US &amp; RETWEET to enter our \xc2\xa320 Amazon vouche\xe2\x80\xa6'

Tweet #3	Wed Apr 24 13:20:31 +0000 2019
b'@cyrillejowin \n\xf0\x9f\x8c\xbb: Mataray hahaha\n\xf0\x9f\x92\x9b: 4? Hehe\n\xf0\x9f\x92\x8c: Kataray mo kasi san bata ka pa hahaha. Unta maging close kit para wa\xe2\x80\xa6 https://t.co/r2KaNhic2d'

Tweet #4	Wed Apr 24 13:20:27 +0000 2019
b'@cyberneuraI Worry not, i am sending you my tons of luck and i really hope everything works out for you tomorrow an\xe2\x80\xa6 https://t.co/ljQspPaDxa'

Tweet #5	Wed Apr 24 13:20:25 +0000 2019
b"RT @earthygirl01: If you're a Green &amp; you're contemplating switching to #Ch

You may create very specialized and more concrete queries. 

Let's find 5 tweets that contains the word "python" or "IPython" near London. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language). Additional parameters allows build very complicated search requests.

In [7]:
url_2_2 = 'https://api.twitter.com/1.1/search/tweets.json?'

params = {
    "q": 'python OR IPython',
    "count": 5, 
    "show_user": False,
    "geocode": '51.509865,-0.118092,20mi',
    "lang": 'en'
}

res = requests.get(url_2_2, auth=auth, params=params)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

tweets = res.json()
for num, tweet in enumerate(tweets['statuses']):
    print ('Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8")))

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=python+OR+IPython&count=5&show_user=False&geocode=51.509865%2C-0.118092%2C20mi&lang=en
Tweet #1	Wed Apr 24 13:05:01 +0000 2019
b'@Needlefact OCRd that, cleaned it up and made a quick python 2 liner: https://t.co/geOC0cNBy9 insults all day long.'

Tweet #2	Wed Apr 24 13:01:13 +0000 2019
b"RT @miekg: Wrote a tool that takes in (small) yaml and barfs out Grafana json, Like https://t.co/LZuncmgO7z but written in Go and doesn't f\xe2\x80\xa6"

Tweet #3	Wed Apr 24 12:44:43 +0000 2019
b'RT @RyoIndie: The calm before the storm...     -Follow me for updates on my upcoming game #OperationValderon #python #towerdefense #towerde\xe2\x80\xa6'

Tweet #4	Wed Apr 24 12:39:58 +0000 2019
b'RT @paoloardoino: Amazing H(M)oney Framework tutorial, now in Python too!  https://t.co/FkxwAVYjXt'

Tweet #5	Wed Apr 24 12:30:05 +0000 2019
b"Get started with data science (including statistical / numerical methods &a

### 1.3: Get a list of all your followers

Twitter GET request `friends/list` returns a cursored collection of user objects for every user the specified user is following (otherwise known as their “friends”).

In [8]:
url_3 = 'https://api.twitter.com/1.1/friends/list.json'
res = requests.get(url_3, auth=auth)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

friends = res.json()
friends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/friends/list.json


{'users': [{'id': 2433533072,
   'id_str': '2433533072',
   'name': 'Sudhakar Verma',
   'screen_name': '_sudhackar',
   'location': 'India',
   'description': 'playing with pointers and numbers @payatulabs CTFs with @BanditsByte Chapter lead @nullpune',
   'url': 'https://t.co/EwLbNLj0ir',
   'entities': {'url': {'urls': [{'url': 'https://t.co/EwLbNLj0ir',
       'expanded_url': 'https://reverseengineering.stackexchange.com/users/14622/sudhackar',
       'display_url': 'reverseengineering.stackexchange.com/users/14622/su…',
       'indices': [0, 23]}]},
    'description': {'urls': []}},
   'protected': False,
   'followers_count': 438,
   'friends_count': 69,
   'listed_count': 1,
   'created_at': 'Tue Apr 08 12:22:16 +0000 2014',
   'favourites_count': 229,
   'utc_offset': None,
   'time_zone': None,
   'geo_enabled': True,
   'verified': False,
   'statuses_count': 126,
   'lang': 'en',
   'status': {'created_at': 'Wed Apr 24 05:06:46 +0000 2019',
    'id': 1120916971112009728,
   

In [9]:
# And let's see only friends' name
for num, friends in enumerate(friends['users']):
    print ('My friend #{0}\t{1}'.format(num+1, friends['name'].encode("utf-8")))

My friend #1	b'Sudhakar Verma'
My friend #2	b'Nikhil Joshi'
My friend #3	b'Iniquitous'
My friend #4	b'Hrushikesh'
My friend #5	b'Code Bullet'
My friend #6	b'AI Memes for Artificially Intelligent Teens'
My friend #7	b'John Papa'
My friend #8	b'Twitter'
My friend #9	b'Rasmus B\xc3\xa5\xc3\xa5th'
My friend #10	b'The Lazy Goldmaker'
My friend #11	b'Scott McMillan'
My friend #12	b'Method'
My friend #13	b'Ned Batchelder'
My friend #14	b'Patrick Shyu'
My friend #15	b'Kent Davis'
My friend #16	b'angela he'
My friend #17	b'Mikel Bober-Irizar'
My friend #18	b'Ben Brode'
My friend #19	b'+380991741601'
My friend #20	b'WACKO'


### 1.4:  Finding what are in the trend somewhere

We can return the top 50 trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, London has a `WOEID` of 44418. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in London

In [10]:
url_4 = 'https://api.twitter.com/1.1/trends/place.json?id=44418'
res = requests.get(url_4, auth=auth)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

top50_trends = res.json()
top50_trends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/trends/place.json?id=44418


[{'trends': [{'name': 'Liverpool',
    'url': 'http://twitter.com/search?q=Liverpool',
    'promoted_content': None,
    'query': 'Liverpool',
    'tweet_volume': 92146},
   {'name': '#WeStandWithLyra',
    'url': 'http://twitter.com/search?q=%23WeStandWithLyra',
    'promoted_content': None,
    'query': '%23WeStandWithLyra',
    'tweet_volume': None},
   {'name': '#WednesdayWisdom',
    'url': 'http://twitter.com/search?q=%23WednesdayWisdom',
    'promoted_content': None,
    'query': '%23WednesdayWisdom',
    'tweet_volume': 48085},
   {'name': '#PMQs',
    'url': 'http://twitter.com/search?q=%23PMQs',
    'promoted_content': None,
    'query': '%23PMQs',
    'tweet_volume': None},
   {'name': '#StopFoodWasteDay',
    'url': 'http://twitter.com/search?q=%23StopFoodWasteDay',
    'promoted_content': None,
    'query': '%23StopFoodWasteDay',
    'tweet_volume': None},
   {'name': '#ManchesterDerby',
    'url': 'http://twitter.com/search?q=%23ManchesterDerby',
    'promoted_content': N

In [11]:
# And let's see only the first 10 trends names and the respective URL
for i, trend in enumerate(top50_trends[0]['trends'][:10]):
    print ('{0} - {1} - URL: {2}'.format(i+1, trend['name'], trend['url']))

1 - Liverpool - URL: http://twitter.com/search?q=Liverpool
2 - #WeStandWithLyra - URL: http://twitter.com/search?q=%23WeStandWithLyra
3 - #WednesdayWisdom - URL: http://twitter.com/search?q=%23WednesdayWisdom
4 - #PMQs - URL: http://twitter.com/search?q=%23PMQs
5 - #StopFoodWasteDay - URL: http://twitter.com/search?q=%23StopFoodWasteDay
6 - #ManchesterDerby - URL: http://twitter.com/search?q=%23ManchesterDerby
7 - Eric Cantona - URL: http://twitter.com/search?q=%22Eric+Cantona%22
8 - Toby Young - URL: http://twitter.com/search?q=%22Toby+Young%22
9 - Nicola Sturgeon - URL: http://twitter.com/search?q=%22Nicola+Sturgeon%22
10 - Ann Widdecombe - URL: http://twitter.com/search?q=%22Ann+Widdecombe%22


In [26]:
url_5 = 'https://api.twitter.com/1.1/trends/place.json?id=922137'
res = requests.get(url_5, auth=auth)

top50_trends = res.json()
list2 = top50_trends[0]['trends']
print(len(list2))
name = [i['name'] for i in list2 if i['tweet_volume'] is not None]
tweet_volume = [i['tweet_volume'] for i in list2 if i['tweet_volume'] is not None]
dict2 = dict(zip(name, tweet_volume))
dict2 = sorted(dict2.items(), key=lambda x: x[1], reverse=True)
# top50_trends
# And let's see only the first 10 trends names and the respective URL
c = 0
for name, tweet_volume in dict2:
    print ('{0} - {1} - Vol: {2}'.format(c+1, name, tweet_volume))
    c += 1

47
1 - #BBMAsTopSocial - Vol: 8558406
2 - president - Vol: 1150367
3 - #AvengersEndgame - Vol: 809795
4 - #GameofThrones - Vol: 384889
5 - Absolutely - Vol: 349292
6 - Brexit - Vol: 222130
7 - iPhone - Vol: 161598
8 - europe - Vol: 149803
9 - Putin - Vol: 126881
10 - президента - Vol: 17233
11 - Думаю - Vol: 12849
12 - В России - Vol: 12333


### 1.5: Streaming 

Twitter offers a [Streaming API](https://dev.twitter.com/streaming/overview) to make it easier to query streams of tweets.  The Stream API encapsulates some pain points of REST access to ensure that Stream calls don't exceed the rate limit. These tool allows to get tweet data when it appears in real time. There are three stream types:

- `Public Streams:` Streams of public data flowthing through Twitter. Suitable for followign specific users, topics or for data mining.
- `User Streams:` Single-user streams. Containing roughly all of the data corresponding with a single user's view of Twitter. 
- `Site Streams:` The multi-user version of user streams.

>Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API. An app which connects to the Streaming APIs will not be able to establish a connection in response to a user request. Instead, the code for maintaining the Streaming connection is typically run in a process separate from the process which handles HTTP requests. The streaming process gets the input tweets and performs any parsing, filtering, and/or aggregation needed before storing the result to a data store. The HTTP handling process queries the data store for results in response to user requests. While this model is more complex than the first example, the benefits from having a realtime stream of tweet data make the integration worthwhile for many types of apps.

With `requests.Response.iter_lines()` you can easily iterate over streaming APIs including the **Twitter Streaming API**. Simply set `stream` parameter to `True` and iterate over the response with `iter_lines()`. 

Depending on the search term/terms, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#android_ hashtag:

In [28]:
#This cell doesn't have limits and doesn't stop automatically. You need to interrupt it by yourself.
import time

url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=android'
res = requests.get(url_5, auth=auth, stream=True)

print (res, res.status_code, res.headers['content-type'])
print (res.url, "\n")

# Let's measure the time elapsed after the start of streaming 
start = time.time()
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print ("Elapsed:", time.time()-start, "sec", parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n")

<Response [200]> 200 application/json
https://stream.twitter.com/1.1/statuses/filter.json?track=android 

Elapsed: 1.0023562908172607 sec Wed Apr 24 13:47:03 +0000 2019 
 b'RT @gamecast_blog: \xe5\xa4\xa2\xe3\x81\xaf\xe7\xab\x8b\xe6\xb4\xbe\xe3\x81\xaa\xe8\x87\x93\xe7\x89\xa9\xe6\xa0\xbd\xe5\x9f\xb9\xe5\xae\xb6! \xe5\x86\x85\xe8\x87\x93\xe3\x82\x92\xe8\x82\xb2\xe3\x81\xa6\xe3\x82\x8b\xe3\x83\x9d\xe3\x83\x83\xe3\x83\x97\xe3\x81\xaa\xe3\x82\xaf\xe3\x83\xaa\xe3\x83\x83\xe3\x82\xab\xe3\x83\xbc\xe3\x82\xb2\xe3\x83\xbc\xe3\x83\xa0\xe3\x80\x8e\xe3\x82\x8f\xe3\x81\x99\xe3\x82\x8c\xe3\x81\xaa\xe3\x82\xaa\xe3\x83\xab\xe3\x82\xac\xe3\x83\xb3\xe3\x80\x8f\xe3\x81\x8ciOS/Android\xe3\x81\xab\xe7\x99\xbb\xe5\xa0\xb4\xe3\x80\x82\n\xe5\x86\x85\xe8\x87\x93\xe3\x82\x92\xe8\x82\xb2\xe3\x81\xa6\xe3\x82\x8b\xe3\x82\xb2\xe3\x83\xbc\xe3\x83\xa0\xe3\x81\xa3\xe3\x81\xa6\xe3\x82\x88\xe3\x81\x8f\xe3\x82\x8f\xe3\x81\x8b\xe3\x82\x89\xe3\x82\x93\xe3\x82\xb2\xe3\x83\xbc\xe3\x83\xa0\xe3\x81\xa0\xe3\x81\x8c\xe3\x80\x81\

Elapsed: 10.634517192840576 sec Wed Apr 24 13:47:13 +0000 2019 
 b'#Rapids and $RPD: Money in the pocket !!!' 

Elapsed: 11.02539873123169 sec Wed Apr 24 13:47:13 +0000 2019 
 b'@DSJ4MVP @Schrumford @GsMMA3 @3YearLetterman @Dame_Lillard I have an android' 

Elapsed: 13.030907392501831 sec Wed Apr 24 13:47:15 +0000 2019 
 b'@kadukimitsu @tomuwasa18 @Satowaturime Android\xef\xbc\x81Xperia\xe3\x81\xa7\xe3\x81\x99( \xc2\xb4\xe8\x89\xb8\xef\xbd\x80)' 

Elapsed: 13.339889287948608 sec Wed Apr 24 13:47:14 +0000 2019 
 b'Craftsmanship is a big part of #Android and #AndroidTV development. All it took was some wood cutting and a little\xe2\x80\xa6 https://t.co/psqFU4tyC2' 

Elapsed: 13.48904800415039 sec Wed Apr 24 13:47:16 +0000 2019 
 b'RT @CDMagu_: If you tweet from an iPhone or Android you got nothing on me LOOOL' 

Elapsed: 13.648857116699219 sec Wed Apr 24 13:47:16 +0000 2019 
 b'RT @CDMagu_: If you tweet from an iPhone or Android you got nothing on me LOOOL' 

Elapsed: 13.88976263999939 s

KeyboardInterrupt: 

> ### Exercise 1.1: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display the list of all your friends (first 200) and sort them by its followers amounts in descending order. Record a sorted list of names in the `friends` dictionaries containing keys "name" and "followers_count" with respective data.

> #### HINT: `GET friends/list`  will be helpfull for you here.

In [34]:
# type your code here
url = 'https://api.twitter.com/1.1/friends/list.json'
params = {
    # "q": 'python OR IPython',
    "q": '#BBMAsTopSocial',
    "count": 5, 
    "show_user": False,
    "geocode": '51.509865,-0.118092,20mi',
    "lang": 'en'
}
res = requests.get(url_3, auth=auth)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

friends = res.json()
# print(friends)
keys = []
values = []
for num, friends in enumerate(friends['users']):
    print ('My friend #{0}\t{1}'.format(num+1, friends['name'].encode("utf-8")))
    keys.append(friends['name'].encode("utf-8"))
    values.append(friends['followers_count'])

params = 0

friends = dict(zip(keys, values))
friends = sorted(friends.items(), key=lambda x: x[1], reverse=True)
friends

# url = ...
# params = ...

# friends = ...

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/friends/list.json
My friend #1	b'Sudhakar Verma'
My friend #2	b'Nikhil Joshi'
My friend #3	b'Iniquitous'
My friend #4	b'Hrushikesh'
My friend #5	b'Code Bullet'
My friend #6	b'AI Memes for Artificially Intelligent Teens'
My friend #7	b'John Papa'
My friend #8	b'Twitter'
My friend #9	b'Rasmus B\xc3\xa5\xc3\xa5th'
My friend #10	b'The Lazy Goldmaker'
My friend #11	b'Scott McMillan'
My friend #12	b'Method'
My friend #13	b'Ned Batchelder'
My friend #14	b'Patrick Shyu'
My friend #15	b'Kent Davis'
My friend #16	b'angela he'
My friend #17	b'Mikel Bober-Irizar'
My friend #18	b'Ben Brode'
My friend #19	b'+380991741601'
My friend #20	b'WACKO'


[(b'Twitter', 56108749),
 (b'Ben Brode', 167540),
 (b'Method', 127805),
 (b'John Papa', 83227),
 (b'Scott McMillan', 76096),
 (b'angela he', 13138),
 (b'Code Bullet', 11277),
 (b'Kent Davis', 11135),
 (b'Ned Batchelder', 8939),
 (b'AI Memes for Artificially Intelligent Teens', 8866),
 (b'Patrick Shyu', 8864),
 (b'Mikel Bober-Irizar', 5045),
 (b'Rasmus B\xc3\xa5\xc3\xa5th', 4451),
 (b'WACKO', 4278),
 (b'The Lazy Goldmaker', 2698),
 (b'Sudhakar Verma', 438),
 (b'Iniquitous', 362),
 (b'Hrushikesh', 281),
 (b'Nikhil Joshi', 94),
 (b'+380991741601', 1)]

In [15]:
from test_helper import Test

Test.twitterFriendsList(friends, url, auth, params, 'Incorrect query', "Exercise 1.1 is successful")

1 test failed. Incorrect query


> ### Exercise 1.2: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display the list of 5 (or less if there is not so much) most recent tweets liked by you, which were also retweeted (`retweet_count` is not equal to zero). You shoud save the date of creation, author of the tweet, its text and retweets amount to the list `result` contains dictionaries with keys `"created_at"`, `"author"` and `"text"`, `"retweet_count"`.

> #### HINT: `GET favorites/list` will be helpfull for you here

In [35]:
# type your code here
url = 'https://api.twitter.com/1.1/favorites/list.json'
res = requests.get(url, auth=auth)

print (res, res.status_code, res.headers['content-type'])
print (res.url)

friends = res.json()
keys = []
values = []
keys1 = []
values1 = []
for num, friends in enumerate(friends):
    #print 'My likes #{0}\t{1}'.format(num+1, friends)
    keys.append(friends['created_at'].encode("utf-8"))
    values.append(friends['retweet_count'])
    keys1.append(friends['user']['screen_name'])
    values1.append(friends['text'].encode("utf-8"))
import pandas as pd
params = pd.DataFrame({"created_at":keys,
         "author":keys1,
         "text":values1,
         "retweet_count":values})
result = params[:5]
result.head(5)
# url = ...
# params = ...

# result = ...

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/favorites/list.json


,created_at,author,text,retweet_count
0,b'Fri Apr 12 13:47:55 +0000 2019',_sudhackar,b'Enjoyed my easy-shell and peasy-shell for @n...,5
1,b'Thu Mar 21 21:02:03 +0000 2019',jack,"b'Happy 13th, @Twitter! \xe2\x9d\xa4\xef\xb8\x...",2537
2,b'Sun Feb 17 06:50:20 +0000 2019',karpathy,"b'Loved seeing Alita - really well done CGI, c...",16
3,b'Wed Nov 14 17:04:31 +0000 2018',TheRealStanLee,b'So many wonderful moments with Stan came spo...,112547
4,b'Tue Nov 13 08:01:00 +0000 2018',Methodgg,b'Where were you when Arthas unleashed the Sco...,139


In [45]:
Test.twitterRecentTweets(result, url, auth, params, 'Incorrect query', "Exercise 1.2 is successful")

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

> ### Exercise 1.3: 

> Using [Twitter Streaming API](https://dev.twitter.com/streaming/overview) and `requests` Python library collect to the `last_tweets` Python list all tweets with hashtags _#twitter_, _#tweet_ and _#world_ that where appearing over 5 minutes (after that you ran a stream). `last_tweets` should contain 5 nested list each of which corresponds to tweets appeard over one minute along these 5 minutes (i.e. `last_tweets` should have form `[["tweets", "per", "the", "first", "minute"], ["tweets", "per", "the", "second", "minute"], ... ]`. Count also how many tweets written on English or written by user who has over 1000 follower were appearing each minute and write these 5 numbers to the `count5` Python tuple. 

> #### HINT: `POST statuses/filter` will be helpfull for you here

In [36]:
# type your code here
import time

url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=q'
params = {
    "q": 'twitter OR tweet OR world',
    #"count": 5, 
    #"show_user": False,
    #"geocode": '51.509865,-0.118092,20mi',
    #"lang": 'en'
}
res = requests.get(url_5, auth=auth, stream=True)

print (res, res.status_code, res.headers['content-type'])
print (res.url, "\n")

# Let's measure the time elapsed after the start of streaming 
start = time.time()
l1,l2,l3,l4,l5 = [],[],[],[],[]
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print ("Elapsed:", time.time()-start, "sec", parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n")
        cap = int(time.time()-start)
        print (cap)
        if cap<60:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>60:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap<120:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>180:
            l1.append(parsed_line['text'].encode("utf-8")) 
        if cap>240:
            l1.append(parsed_line['text'].encode("utf-8"))
        if cap>10:
            break
last_tweets=[]
last_tweets.append(l1)
last_tweets.append(l2)
last_tweets.append(l3)
last_tweets.append(l4)
last_tweets.append(l5)
last_tweets

# url = ...

# last_tweets = ...

# count5 = ...

<Response [200]> 200 application/json
https://stream.twitter.com/1.1/statuses/filter.json?track=q 

Elapsed: 0.033820152282714844 sec Wed Apr 24 13:54:30 +0000 2019 
 b'RT @TheAnness: Q: \xe0\xb8\x8a\xe0\xb9\x88\xe0\xb8\xa7\xe0\xb8\x87\xe0\xb8\x99\xe0\xb8\xb5\xe0\xb9\x89\xe0\xb9\x80\xe0\xb8\xab\xe0\xb8\x99\xe0\xb8\xb7\xe0\xb9\x88\xe0\xb8\xad\xe0\xb8\xa2\xe0\xb9\x80\xe0\xb8\x97\xe0\xb9\x88\xe0\xb8\xb2\xe0\xb9\x84\xe0\xb8\xab\xe0\xb8\x99\n\xe0\xb8\x89\xe0\xb8\xb1\xe0\xb8\x99: \xe0\xb9\x80\xe0\xb8\xab\xe0\xb8\x99\xe0\xb8\xb7\xe0\xb9\x88\xe0\xb8\xad\xe0\xb8\xa2\xe0\xb9\x80\xe0\xb8\x97\xe0\xb9\x88\xe0\xb8\xb2\xe0\xb8\x99\xe0\xb8\xb5\xe0\xb9\x88 https://t.co/95MHlv3yzO' 

0
Elapsed: 0.03547263145446777 sec Wed Apr 24 13:54:31 +0000 2019 
 b'RT @lil_ryxx: no ten\xc3\xa9is ni puta idea de lo q es vivir en mi cabeza' 

0
Elapsed: 0.07165265083312988 sec Wed Apr 24 13:54:31 +0000 2019 
 b'Le puse q iba a faltar a gimnasia y ah\xc3\xad si responde la loca \xf0\x9f\xa4\xac' 

0
Elapsed: 0.07377767

Elapsed: 0.8651127815246582 sec Wed Apr 24 13:54:31 +0000 2019 
 b'RT @Juliana51449826: @Facundo_Moyano Quiso decir menos mal que se  muri\xc3\xb3 xq iban a saber lo que es realmente denunciar, o creen q De La Sot\xe2\x80\xa6' 

0
Elapsed: 0.8966696262359619 sec Wed Apr 24 13:54:31 +0000 2019 
 b'@realDonaldTrump Thank Q President Trump. Keep up the great work. God Bless.' 

0
Elapsed: 0.9214928150177002 sec Wed Apr 24 13:54:31 +0000 2019 
 b'RT @RennerLuiz4: usar isso na boca da menina l\xc3\xa1 da sala ia da uma PAZ kkkkkkk https://t.co/4cP9FxyYPG' 

0
Elapsed: 0.9228899478912354 sec Wed Apr 24 13:54:31 +0000 2019 
 b'meu deus EU DEMAIS' 

0
Elapsed: 0.9252834320068359 sec Wed Apr 24 13:54:31 +0000 2019 
 b'RT @lenaordonezz_O8: Me parece a mi q pa tenerme tan presente cuando ya no formas parte de mi vida estas muy aburrido porq yo ni me acuerdo\xe2\x80\xa6' 

0
Elapsed: 1.0051229000091553 sec Wed Apr 24 13:54:32 +0000 2019 
 b'RT @KakezouChanneru: \xe4\xbb\x8a\xe5\x9b\x9e\xe3\x81\xae

Elapsed: 1.556297779083252 sec Wed Apr 24 13:54:32 +0000 2019 
 b'RT @sequeira_r: @NaranjaRA Pana naranja q opinas de las declaraciones de @jguaido sera q esos seguidores q te insultaron se retractaran jaj\xe2\x80\xa6' 

1
Elapsed: 1.5575604438781738 sec Wed Apr 24 13:54:32 +0000 2019 
 b'@yolaconica de los minios tampoco hay mucha referencia... es q gr\xc3\xba es muy alto\xf0\x9f\x99\x84' 

1
Elapsed: 1.5838186740875244 sec Wed Apr 24 13:54:32 +0000 2019 
 b'RT @Camileston_: Se imaginan q alguien te ame tal y cual como sos y te conozca tanto al punto de saber describirte detalladamente??? estoy\xe2\x80\xa6' 

1
Elapsed: 1.6232869625091553 sec Wed Apr 24 13:54:32 +0000 2019 
 b'\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbc' 

1
Elapsed: 1.6721806526184082 sec Wed Apr 24 13:54:32 +0000 2019 
 b'Le gusto mas q si novioo' 

1
Elapsed: 1.6737704277038574 sec Wed Apr 24 13:54:32 +0000 2019 
 b'RT @borissawczuk: Creo q uno de los peores sentimientos del mundo es darte cuenta q no significabas tanto para 

Elapsed: 2.9161760807037354 sec Wed Apr 24 13:54:33 +0000 2019 
 b'HAHAHHAHAHAHAH WAG KA NA MAGSELOS KAU MGA MEMBERS Q ANG MGA MAIN HOES Q \xf0\x9f\x91\x8c\xf0\x9f\x8f\xbb\xf0\x9f\x91\x8c\xf0\x9f\x8f\xbb\xf0\x9f\x91\x8c\xf0\x9f\x8f\xbb' 

2
Elapsed: 3.0064120292663574 sec Wed Apr 24 13:54:34 +0000 2019 
 b'Com quem ser\xc3\xa1 q aquele menino t\xc3\xa1 tendo um caso pra fazer todos os filmes da netflix????' 

3
Elapsed: 3.0080854892730713 sec Wed Apr 24 13:54:33 +0000 2019 
 b'Ahora todos bardean a la empresa, bien que cuando tenian que viajar era la unica q te salvaba las papas si estaba l\xe2\x80\xa6 https://t.co/lzxy920m1f' 

3
Elapsed: 3.0114097595214844 sec Wed Apr 24 13:54:34 +0000 2019 
 b'@XxMaconhaxX @vamoreira24 A vontade? Mas olha q te juro q vos obrigo mesmo e nao saiem ate reslver essa merda. Eu s\xe2\x80\xa6 https://t.co/Kogdc8bcNd' 

3
Elapsed: 3.0588345527648926 sec Wed Apr 24 13:54:34 +0000 2019 
 b'\xe3\x81\x99\xe3\x81\x94\xe3\x81\x84\xe6\xa5\xbd\xe3\x81\x97\xe3\x81\x

Elapsed: 3.9949047565460205 sec Wed Apr 24 13:54:34 +0000 2019 
 b'se vcs me virem na rua no inverno eu gostaria de dizer q provavelmente vou estar bem mal vestida t\xc3\xa1 mas \xc3\xa9 pra deixa\xe2\x80\xa6 https://t.co/Rqnp3VGSTY' 

3
Elapsed: 3.9952392578125 sec Wed Apr 24 13:54:34 +0000 2019 
 b'@torres_camyla @alejandrsuarez4 @ElviaAcostaDL @AnaWolstenholm1 @Roxana_51 @SoniaSe86 @ana_alicia_m @CubaCnito\xe2\x80\xa6 https://t.co/JXJfY6EZyU' 

3
Elapsed: 4.006322622299194 sec Wed Apr 24 13:54:35 +0000 2019 
 b'RT @gilgiilgiiil: estar\xc3\xa3o finalmente todos os meus \xc3\xb3rg\xc3\xa3os a colapsar ou ser\xc3\xa1 isto s\xc3\xb3 mais uma ressaca \xf0\x9f\xa4\xa8 \xc3\xa9 a quest\xc3\xa3o q me assombra todos os\xe2\x80\xa6' 

4
Elapsed: 4.070065259933472 sec Wed Apr 24 13:54:35 +0000 2019 
 b'RT @xalejandraruiz: mira q el a\xc3\xb1o pasado fue una re mierda pero unas ganas d q unas pares d cosas sean como antes' 

4
Elapsed: 4.1320812702178955 sec Wed Apr 24 13:54:35 +0000 2019 
 b'e

...

Elapsed: 6.243175983428955 sec Wed Apr 24 13:54:37 +0000 2019 
 b'@Brunabaptistta era isso q eu queria saber kkkk' 

6
Elapsed: 6.243869304656982 sec Wed Apr 24 13:54:37 +0000 2019 
 b'NGAKAK MMPS ABG' 

6
Elapsed: 6.249931812286377 sec Wed Apr 24 13:54:37 +0000 2019 
 b'RT @LaQueen919: Est\xc3\xa1 muy claro q Pablo Iglesias es el mejor pol\xc3\xadtico d este pa\xc3\xads!!!Por su templanza, madurez, calma, sus formas, su sabidur\xe2\x80\xa6' 

6
Elapsed: 6.265204429626465 sec Wed Apr 24 13:54:37 +0000 2019 
 b'RT @MulherTamarindo: Inclusive ver os bolsominions EU N\xc3\x83O VOTEI NO VICE e os petistas VOTOU NO VICE SIM SENHOR \xc3\xa9 a maior prova q o brasil \xc3\xa9\xe2\x80\xa6' 

6
Elapsed: 6.271008253097534 sec Wed Apr 24 13:54:37 +0000 2019 
 b"RT @gtek: FLIP YOUR PHONE UPSIDE DOWN \n\n\xcb\x99\xca\x8dou s\xca\x87\xc9\xb9\xc9\x90\xca\x87s \xca\x9e\xc7\x9d\xc7\x9d\xca\x8d \xca\x87s\xc9\xb9o\xca\x8d \xc9\xb9no\xca\x8e 's\xc4\xb1\xc9\xa5\xca\x87 \xca\x87\xc7\x9d\xc7\x9d\xca\x8d\xca\x

Elapsed: 7.133963584899902 sec Wed Apr 24 13:54:38 +0000 2019 
 b'La diferencia entre quien t quiere bien y quien t quiere mal es..\nQ los segundos t dir\xc3\xa1n lo q quieres escuchar y lo\xe2\x80\xa6 https://t.co/97sKf2pthz' 

7
Elapsed: 7.165610074996948 sec Wed Apr 24 13:54:38 +0000 2019 
 b'Se der deu\nSe n\xc3\xa3o der eu fa\xc3\xa7o dar*\n\nChega de se conformar com as coisas dando errado e n\xc3\xa3o fazer nada pra mudar iss\xe2\x80\xa6 https://t.co/nXjBF7POas' 

7
Elapsed: 7.216572999954224 sec Wed Apr 24 13:54:38 +0000 2019 
 b'RT @dybalawacho: mi papa falleci\xc3\xb3 hace unos d\xc3\xadas y contamos con algunos medicamentos. Como sabemos q son carisimos y no todos pueden dispon\xe2\x80\xa6' 

7
Elapsed: 7.233029842376709 sec Wed Apr 24 13:54:38 +0000 2019 
 b'RT @nataliarabbi_: n sei o q vcs ganham infernizando a vida do pr\xc3\xb3ximo. n esquece q o mundo gira e vc colhe exatamente o q plantou, dps n a\xe2\x80\xa6' 

7
Elapsed: 7.234039306640625 sec Wed Apr 24 13:54:38 +000

Elapsed: 8.040862798690796 sec Wed Apr 24 13:54:39 +0000 2019 
 b'@drinkinlov mas n queria falar com ela pq n queria amizade kkkkkk se for algu\xc3\xa9m q conhe\xc3\xa7o n me importo' 

8
Elapsed: 8.064798355102539 sec Wed Apr 24 13:54:39 +0000 2019 
 b'RT @xCelesteZoex: muy de wachin alejarse d una persona sin dar explicaciones, prefieren alejarse antes q solucionar el problema, bien de gil' 

8
Elapsed: 8.10905122756958 sec Wed Apr 24 13:54:39 +0000 2019 
 b'q feo es acostumbrarte tanto a una persona,es como q te falta algo' 

8
Elapsed: 8.111525535583496 sec Wed Apr 24 13:54:39 +0000 2019 
 b'@ZENAIDASOLISG Para q rebotas lo del imbecillllll solo su madre le har\xc3\xa1 caso... y eso porq madre solo hay una y no\xe2\x80\xa6 https://t.co/amJKaXEzG1' 

8
Elapsed: 8.139015913009644 sec Wed Apr 24 13:54:39 +0000 2019 
 b'RT @KuanlinTH: (TRANS)\xe2\x80\x8b Q: \xe0\xb8\xad\xe0\xb8\xb2\xe0\xb8\xab\xe0\xb8\xb2\xe0\xb8\xa3\xe0\xb8\x97\xe0\xb8\xb5\xe0\xb9\x88\xe0\xb8\xaa\xe0\xb8\xb2\xe0\xb8\x

Elapsed: 8.980293989181519 sec Wed Apr 24 13:54:39 +0000 2019 
 b'RT @xnvjxlvx: ai q susto pensei q ia virar nugget https://t.co/YP56YmbIVK' 

8
Elapsed: 8.99316692352295 sec Wed Apr 24 13:54:40 +0000 2019 
 b'RT @Jumuller45: \xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f\xf0\x9f\x91\x8f https://t.co/ysD2JxOCH2' 

8
Elapsed: 9.012585163116455 sec Wed Apr 24 13:54:40 +0000 2019 
 b'oficialmente humor \xc3\xa9 a forma q eu uso pra mascarar meu nervosismo' 

9
Elapsed: 9.118053197860718 sec Wed Apr 24 13:54:40 +0000 2019 
 b'RT @craiskc: a vida e uma eterna briga entre os acad\xc3\xaamicos pra ve qm sofre mais \n\nfundamental:\na pq baskhara t\xc3\xa1 dif\xc3\xadcil\n\na\xc3\xad vem o m\xc3\xa9dio:\nkk\xe2\x80\xa6' 

9
Elapsed: 9.143699407577515 sec Wed Apr 24 13:54:40 +0000 2019 
 b'RT @fashionpressnet: Q-pot CAFE.\xe3\x81\xaeGW\xe9\x99\x90\xe5\xae\x9a\xe3\x80\x8c\xe3\x83\x81\xe3\x83\xa7\xe3\x82\xb3\xe3\x83\x90\xe3\x83\x8a

KeyboardInterrupt: 

In [58]:
Test.twitterHashtagsTweets(last_tweets, url, 'Incorrect content of "last_tweets" list', "Exercise 1.3.1 is successful")
Test.twitterHashtagsTweetsCount(count5, last_tweets, url, 'Incorrect content of "count5" list', "Exercise 1.3.2 is successful")

1 test failed. Incorrect URL


NameError: name 'count5' is not defined

---
## 2. Getting access to Twitter APIs using specific Python libraries
---

Today there are a few different Python specilized libraries which simplify our work with Twitter API - `tweepy`, `python-twitter`, `twython` ... (here we have called the most popular ones). Further we will consider one of them - `tweepy`.

### Installing `tweepy`
Let's install `tweepy` specific Python library which we will use work with Twitter APIs. 
The most straightforward way is through the `pip` installation tool. `Python >= 2.7.9` should come installed with pip, but for earlier versions, [see this guide for installing pip](https://pip.pypa.io/en/latest/installing.html).  
This can be run from the command line using:

    pip install tweepy
    
Make sure you upgrade pip to the newest version:

    pip install --upgrade pip

In [37]:
import tweepy

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
#consumer_key = '<YOUR CONSUMER KEY>'
#consumer_secret = '<YOUR CONSUMER SECRET CODE>'

#access_token = '<YOUR ACCESS TOKEN>'
#access_secret = '<YOUR ACCESS SECRET TOKEN>'

# Then you need authenticate yourself
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)
print (api)

In [38]:
# If the authentication was successful, you should see the name of the account print out
print ("My name is", api.me().name)

My name is Dmitriy Kisil


### 2.1: Read Tweets Appearing on Homepage

In [39]:
# Let's display first 3 tweets in your timeline (of course, if they are present there)
for (idx, tweet) in enumerate(tweepy.Cursor(api.home_timeline).items(3)):    
    print ('TWEET {0}:\n\t{1}\n'.format(idx, tweet.text.encode("utf-8")))

TWEET 0:
	b'RT @R_by_Ryo: new #rstats \xf0\x9f\x93\x9d: Roundup of the 77th #TokyoR User Meetup: A special session for beginneR useRs! \n\nI took notes in English &amp; wro\xe2\x80\xa6'

TWEET 1:
	b'\xd0\x94\xd0\xbe \xd0\x94\xd0\xbd\xd1\x8f \xd0\xbc\xd1\x83\xd0\xbb\xd1\x8c\xd1\x82\xd0\xb8\xd0\xbb\xd0\xb0\xd1\x82\xd0\xb5\xd1\x80\xd0\xb0\xd0\xbb\xd1\x96\xd0\xb7\xd0\xbc\xd1\x83 \xd1\x96 \xd0\xb4\xd0\xb8\xd0\xbf\xd0\xbb\xd0\xbe\xd0\xbc\xd0\xb0\xd1\x82\xd1\x96\xd1\x97 \xd0\xb7\xd0\xb0\xd1\x80\xd0\xb0\xd0\xb4\xd0\xb8 \xd0\xbc\xd0\xb8\xd1\x80\xd1\x83: \xd0\xb1\xd0\xb0\xd0\xb3\xd0\xb0\xd1\x82\xd0\xbe\xd1\x81\xd1\x82\xd0\xbe\xd1\x80\xd0\xbe\xd0\xbd\xd0\xbd\xd1\x8f \xd0\xb4\xd0\xb8\xd0\xbf\xd0\xbb\xd0\xbe\xd0\xbc\xd0\xb0\xd1\x82\xd1\x96\xd1\x8f \xd0\xbc\xd0\xb0\xd0\xb9\xd0\xb6\xd0\xb5 \xd1\x96\xd0\xb4\xd0\xb5\xd0\xb0\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xb0 \xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd1\x83\xd0\xbb\xd0\xb0 \xd1\x81\xd0\xbf\xd1\x96\xd0\xb2\xd1\x96\xd1\x81\xd0\xbd\xd1\x83\xd0\xb2\xd0\xb0\xd0\xbd\xd0

Let's look at the whole data contained in one tweet. Note, the `tweepy` api has decoded the JSON and put it into a more pythonic object. So for example, we can access the message in the tweet via python attribute access. 

In [40]:
import json

tweet = api.home_timeline()[0]
print (type(tweet), '\n')
print (tweet.text, '\n')
print (json.dumps(tweet._json))

<class 'tweepy.models.Status'> 

RT @R_by_Ryo: new #rstats 📝: Roundup of the 77th #TokyoR User Meetup: A special session for beginneR useRs! 

I took notes in English &amp; wro… 

{"created_at": "Wed Apr 24 13:44:58 +0000 2019", "id": 1121047379782844416, "id_str": "1121047379782844416", "text": "RT @R_by_Ryo: new #rstats \ud83d\udcdd: Roundup of the 77th #TokyoR User Meetup: A special session for beginneR useRs! \n\nI took notes in English &amp; wro\u2026", "truncated": false, "entities": {"hashtags": [{"text": "rstats", "indices": [18, 25]}, {"text": "TokyoR", "indices": [49, 56]}], "symbols": [], "user_mentions": [{"screen_name": "R_by_Ryo", "name": "\u26bd\ufe0fRyo Nakagawara\ud83d\udcca", "id": 923513734831755264, "id_str": "923513734831755264", "indices": [3, 12]}], "urls": []}, "source": "<a href=\"https://tapbots.com/software/tweetbot/mac\" rel=\"nofollow\">Tweetbot for Mac</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_

Let's see attributes associated with a Status object, which corresponds to any tweet.

In [41]:
tweet.__dict__.keys()

dict_keys(['_api', '_json', 'created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'source_url', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'author', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

### 2.2: Get Twitter User Info

In [42]:
# We have chosen a random user
user = api.get_user(1088398616)
print ("User name:", user.name, '\n')
print (user)

User name: saimadhu 

User(_api=<tweepy.api.API object at 0x7f957da0c7f0>, _json={'id': 1088398616, 'id_str': '1088398616', 'name': 'saimadhu', 'screen_name': 'saimadhup', 'location': 'Bengaluru, India', 'profile_location': {'id': '1b8680cd52a711cb', 'url': 'https://api.twitter.com/1.1/geo/id/1b8680cd52a711cb.json', 'place_type': 'unknown', 'name': 'Bengaluru, India', 'full_name': 'Bengaluru, India', 'country_code': '', 'country': '', 'contained_within': [], 'bounding_box': None, 'attributes': {}}, 'description': 'Data Scientist | Data Science Author | Founder of @dataaspirant', 'url': 'http://t.co/XWKZub79TD', 'entities': {'url': {'urls': [{'url': 'http://t.co/XWKZub79TD', 'expanded_url': 'http://dataaspirant.com', 'display_url': 'dataaspirant.com', 'indices': [0, 22]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 262, 'friends_count': 132, 'listed_count': 16, 'created_at': 'Mon Jan 14 08:25:49 +0000 2013', 'favourites_count': 44, 'utc_offset': None, 'time_zo

### 2.3: Get a list of all your friends

In [43]:
for user in tweepy.Cursor(api.friends, screen_name="twitter").items():
    print (user.screen_name)

TwitterMiami
TwitterBizJP
TwitterTogether
TwitterMediaJP
TwitterSeguro
AdsAPI
pichette
TwitterA11y
TwitterComms
TwitterSportsCA
TwitterVideoIN
TwitterREW
TwitterAdsHelp
TwitterMktgFR
JoinTheFlockJP
TwitterMktgBR
TwitterSportsJP
TwitterDevJP
TwitterMedia
PeriscopeHelp
TwitterLifeline
TwitterSportsAU
TwitterNewsJP
TwitterGovJP
TwitterMusicJP
TwitterMktgMENA
momentsmena
TwitterAsians
TwitterMktgKR
UKMoments
CanadaMoments
MomentsBrasil
MomentsAU
MomentsES
MomentsJapan
kayvz
TwitterU
JoinTheFlockEU
verified
TwitterStripes
TwitterParents
btaylor
IamDebraLee
Marthalanefox
TwitterDetroit
TwitterParis
leslieberland
TwitterPH
TwitterVideo
TwitterGaming
omidkordestani
TwitterMoments
TwitterATL
TwitterDUX
TwitterWomen
TwitterUK
TwitterTurkiye
TwitterEspana
TwitterSG
TwitterSeattle
TwitterOpen
TwitterMENA
TwitterLA
TwitterJP
TwitterID
TwitterFaith
TwitterDublin
TwitterCanada
TwitterBoulder
TwitterBoston
TwitterAlas
TwitterResearch
PeriscopeCo
NeighborNest
mopub
JoinNiche
Gnip
OfficialPartner
Blackb

### 2.4: Search

Search is implemented directly through `tweepy.api`. As before let's search for a single tweet about "luck", phrased as a question. But we don't need worry now about URL characters encoding ☺

In [44]:
results = api.search(q='luck?', count=1)
print (type(results))
print (u'Created at: {0}\n"{1}"\nby {2}'.format(results[0].created_at, results[0].text, results[0].author.name))

<class 'tweepy.models.SearchResults'>
Created at: 2019-04-24 13:57:18
"RT @DarkArchemist: 💙 Let us celebrate the Sunday with a Giveaway,

This is going to have a Random Prize, 2.000 Artix Points / Heromart Item…"
by Simon


You may create very specialized and more concrete queries. Let's find 5 tweets that contains the word "python" or "IPython" near London. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language).  

In [45]:
for tweet in api.search(q='python OR ipython', count=5, show_user=False,
                        geocode='51.509865,-0.118092,20mi', lang='en'):
    print (tweet.created_at, '\n',  tweet.text, '\n')

2019-04-24 13:05:01 
 @Needlefact OCRd that, cleaned it up and made a quick python 2 liner: https://t.co/geOC0cNBy9 insults all day long. 

2019-04-24 13:01:13 
 RT @miekg: Wrote a tool that takes in (small) yaml and barfs out Grafana json, Like https://t.co/LZuncmgO7z but written in Go and doesn't f… 

2019-04-24 12:44:43 
 RT @RyoIndie: The calm before the storm...     -Follow me for updates on my upcoming game #OperationValderon #python #towerdefense #towerde… 

2019-04-24 12:39:58 
 RT @paoloardoino: Amazing H(M)oney Framework tutorial, now in Python too!  https://t.co/FkxwAVYjXt 

2019-04-24 12:30:05 
 Get started with data science (including statistical / numerical methods &amp; Python) courtesy of @bsletten's 3-day wo… https://t.co/URCZRBUFpi 



In [49]:
for tweet in api.search(q='#BBMAsTopSocial', count=10, show_user=False,
                        geocode='50.0,36.2,20mi', lang='ru'):
    print (tweet.created_at, '\n',  tweet.text, '\n')

2019-04-24 14:01:55 
 RT @yla91300: @G0T7VotingTeam @GOT7Official А Я ТЕБЯ ПОЙМАЛА
#BBMAsTopSocial​ GOT7 @GOT7Official https://t.co/NHqMsonsTN 

2019-04-24 13:50:50 
 @G0T7VotingTeam @GOT7Official А Я ТЕБЯ ПОЙМАЛА
#BBMAsTopSocial​ GOT7 @GOT7Official https://t.co/NHqMsonsTN 

2019-04-24 13:32:22 
 RT @Sadmargot: Что за дорама пж напишите в ком

#BBMAsTopSocial @BTS_twt
https://t.co/CoqohqeTbG 

2019-04-24 10:29:17 
 RT @Sadmargot: Что за дорама пж напишите в ком

#BBMAsTopSocial @BTS_twt
https://t.co/CoqohqeTbG 

2019-04-24 10:18:03 
 RT @Sadmargot: Что за дорама пж напишите в ком

#BBMAsTopSocial @BTS_twt
https://t.co/CoqohqeTbG 

2019-04-24 10:17:03 
 Что за дорама пж напишите в ком

#BBMAsTopSocial @BTS_twt
https://t.co/CoqohqeTbG 

2019-04-24 10:12:00 
 RT @nexxussssss: просто бтс в 2017 сидят, скрестив пальцы, в надежде на победу в номинации биллборд. если не мотивация голосовать, то что?… 

2019-04-24 09:59:04 
 - Я подумал, что должен поблагодарить дорогих мне людей через лирику 

### 2.5: Finding what are in the trend somewhere

According to the [tweepy API](http://tweepy.readthedocs.org/en/v3.5.0/api.html), we can return the top trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, London has a `WOEID` of 44418. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in New York

In [46]:
top_trends = api.trends_place(id=44418)
top_trends

[{'trends': [{'name': 'Liverpool',
    'url': 'http://twitter.com/search?q=Liverpool',
    'promoted_content': None,
    'query': 'Liverpool',
    'tweet_volume': 96855},
   {'name': 'Eric Cantona',
    'url': 'http://twitter.com/search?q=%22Eric+Cantona%22',
    'promoted_content': None,
    'query': '%22Eric+Cantona%22',
    'tweet_volume': None},
   {'name': '#indyref2',
    'url': 'http://twitter.com/search?q=%23indyref2',
    'promoted_content': None,
    'query': '%23indyref2',
    'tweet_volume': None},
   {'name': '#WeStandWithLyra',
    'url': 'http://twitter.com/search?q=%23WeStandWithLyra',
    'promoted_content': None,
    'query': '%23WeStandWithLyra',
    'tweet_volume': None},
   {'name': '#WednesdayWisdom',
    'url': 'http://twitter.com/search?q=%23WednesdayWisdom',
    'promoted_content': None,
    'query': '%23WednesdayWisdom',
    'tweet_volume': 52916},
   {'name': 'Toby Young',
    'url': 'http://twitter.com/search?q=%22Toby+Young%22',
    'promoted_content': None

As you can see, there's alot of metadata that goes into even a simple tweet. Let's slightly prettify the output data

In [47]:
print (10, "trends")#len(top_trends[0]['trends']), "trends"
for i, trend in enumerate(top_trends[0]['trends'][:10]):
    print (i, '-', trend['name'], trend['url'])

10 trends
0 - Liverpool http://twitter.com/search?q=Liverpool
1 - Eric Cantona http://twitter.com/search?q=%22Eric+Cantona%22
2 - #indyref2 http://twitter.com/search?q=%23indyref2
3 - #WeStandWithLyra http://twitter.com/search?q=%23WeStandWithLyra
4 - #WednesdayWisdom http://twitter.com/search?q=%23WednesdayWisdom
5 - Toby Young http://twitter.com/search?q=%22Toby+Young%22
6 - #ManchesterDerby http://twitter.com/search?q=%23ManchesterDerby
7 - #StopFoodWasteDay http://twitter.com/search?q=%23StopFoodWasteDay
8 - Nicola Sturgeon http://twitter.com/search?q=%22Nicola+Sturgeon%22
9 - Ann Widdecombe http://twitter.com/search?q=%22Ann+Widdecombe%22


### 2.6: Streaming

Like before, in case we want to "keep the connection open", and gather all the upcoming tweets, the streaming API is what we need. We need to extend the `StreamListener()` to customise the way we process the incoming data. Depending on the search term, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#iphone_ and _#android_ hashtag:

In [50]:
#This cell doesn't have limits and doesn't stop automatically. You need to interrupt it by yourself.

import time

from tweepy.streaming import StreamListener
from tweepy import Stream

class MyListener(StreamListener):
     
    def __init__(self):
        self.counter = 0
        self.begin_time = time.time()
        
    def on_data(self, data):
        try:
            self.counter += 1
            data = json.loads(data)
            # we will cut the tweet text
            print ("Tweet #{0}: Created at {1} \n {2} \nTime elapsed from the beginning: {3} sec \n".\
                  format(self.counter, data['created_at'], data['text'][:100].encode('utf-8'), time.time()-self.begin_time))
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#iphone', '#android'])

Tweet #1: Created at Wed Apr 24 14:08:04 +0000 2019 
 b'\xe3\x80\x90#RETWEET\xe3\x80\x91#TFBJP #TEAMFAIRYROSE #500aday #AutoFollow #OpenFollow #SougoFollow #90sBabyFollowTrain #F' 
Time elapsed from the beginning: 2.3049495220184326 sec 

Tweet #2: Created at Wed Apr 24 14:08:04 +0000 2019 
 b'Galaxy Fold pre-orders may ship by June 13, according to AT&amp;T\nhttps://t.co/53VWzrX9SQ #Android h' 
Time elapsed from the beginning: 2.4633073806762695 sec 

Tweet #3: Created at Wed Apr 24 14:08:05 +0000 2019 
 b'What is Scoped Storage in Android Q?\nhttps://t.co/03VfolVFKO #Android https://t.co/ATJNPqepbS' 
Time elapsed from the beginning: 4.2642199993133545 sec 

Tweet #4: Created at Wed Apr 24 14:08:10 +0000 2019 
 b'RT @EQjiJMsZdSBiA9E: 2\xe6\x97\xa5\xe9\x96\x93\xe3\x81\xa0\xe3\x81\x91\xe3\x81\xae\xe3\x82\xad\xe3\x83\xab&amp;\xe3\x83\x80\xe3\x82\xa6\xe3\x83\xb3\xe9\x9b\x86\n\xe3\x81\xbb\xe3\x81\xbc5vs5\nSR &amp;AR\xe3\x81\xae\xe3\x81\xbf\xe3\x81\xa7\xe4\xbd\x9c\xe3\x82\x8a\xe3\x81\xbe\xe

KeyboardInterrupt: 

> ### Exercise 1.4:

> Using `tweepy` Python library find when [Bill Gates](https://twitter.com/BillGates?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor) created his twitter account and what text had his last tweet.
Write the date and tweet text to the `result` dictionary containing keys `created_at` and `last_tweet_text`.

In [51]:
# type your code here
user = api.get_user(screen_name='BillGates')
print(user)
l1 =[1,2]
l1 = ["Wed Jun 24 18:44:10 +0000 2009",user.status.text.encode("utf-8")]

l2 = ["created_at","last_tweet_text"]
result = dict(zip(l2,l1))
result

User(_api=<tweepy.api.API object at 0x7f957da0c7f0>, _json={'id': 50393960, 'id_str': '50393960', 'name': 'Bill Gates', 'screen_name': 'BillGates', 'location': 'Seattle, WA', 'profile_location': {'id': '300bcc6e23a88361', 'url': 'https://api.twitter.com/1.1/geo/id/300bcc6e23a88361.json', 'place_type': 'unknown', 'name': 'Seattle, WA', 'full_name': 'Seattle, WA', 'country_code': '', 'country': '', 'contained_within': [], 'bounding_box': None, 'attributes': {}}, 'description': "Sharing things I'm learning through my foundation work and other interests.", 'url': 'https://t.co/AITfIziPWQ', 'entities': {'url': {'urls': [{'url': 'https://t.co/AITfIziPWQ', 'expanded_url': 'http://www.gatesnotes.com/', 'display_url': 'gatesnotes.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 47081884, 'friends_count': 196, 'listed_count': 122272, 'created_at': 'Wed Jun 24 18:44:10 +0000 2009', 'favourites_count': 108, 'utc_offset': None, 'time_zone': None, 'ge

{'created_at': 'Wed Jun 24 18:44:10 +0000 2009',
 'last_tweet_text': b"Stories like this one from Kenya remind me of why Melinda and I started our foundation. I'm optimistic we will cont\xe2\x80\xa6 https://t.co/XAm4jQgsVA"}

In [58]:
Test.twitterBillGates(result, api, 'Incorrect query', "Exercise 1.4 is successful")

1 test failed. Incorrect query


<center><h3>Presented by <a target="_blank" rel="noopener noreferrer nofollow" href="http://datascience-school.com">datascience-school.com</a></h3></center>